In [34]:
import pandas as pd
import requests
import unicodedata
from bs4 import BeautifulSoup as soup
from datetime import datetime

raw_path = '../raw'

In [39]:
# df = pd.read_excel(raw_path + "/HABV_93_Ehrenamtskartei.xlsx", names=['signatur','nachname','vorname','geburt','tod','umfang'], dtype={'signatur':'string','nachname':'string','vorname':'string','geburt':'string','tod':'string','umfang':'string',})
df = pd.read_csv(raw_path + "/ehrenamtskartei.csv", dtype={'signatur':'string','nachname':'string','vorname':'string','geburt':'string','tod':'string','umfang':'string',})
df = df.apply(lambda x: x.str.strip() if (x.dtype == "object") or (x.dtype == "string") else x)

# GND-Suche

In [9]:

base_url = 'https://services.dnb.de/sru/authorities?'
params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
         }

In [17]:

def gnd_abfrage(row):
    params.update({'query': f'PER = "{row.nachname}, {row.vorname}" AND BBG=Tp*'})
    response = requests.get(base_url, params=params)
    response_xml = soup(response.content, features="xml")    
    try:
        matches = list()
        for record in response_xml.find_all('record', {'type':'Authority'}):
            record_match = dict()
            #idn bestimmen
            try:
                record_match['idn'] = record.find('controlfield', {'tag': '001'}).string.strip()
            except:
                record_match['idn'] = None

            # exaktes datum datx finden
            try:
                datx = record.find('subfield', {'code': '4'}, string="datx").parent.find('subfield', {'code': 'a'}).string
            except:
                datx = ''
            
            # jahresdatum in 100 finden
            try:
                datl_100 = record.find('datafield', {'tag': '100'}).find('subfield', {'code': 'd'}).string
            except:
                datl_100 = ''
            
            # jahresdatum in 548 datl finden
            try:
                datl_548 = record.find('subfield', {'code': '4'}, string="datl").parent.find('subfield', {'code': 'a'}).string
            except:
                datl_548 = ''

            #setzen von datum_match, wenn geburtsjahre übereinstimmen

            if datx == f"{row.geburt}-{row.tod}":
                record_match['datum_match'] = True
            elif datl_100.split('-')[0] == row.geburt[-4] and datl_100.split('-')[1] == row.tod[-4]:
                record_match['datum_match'] = True
            elif datl_548.split('-')[0] == row.geburt[-4] and datl_548.split('-')[1] == row.tod[-4]:
                record_match['datum_match'] = True
            else:
                record_match['datum_match'] = False

            matches.append(record_match)
        
        return matches
    except Exception as e:
        return "fehler %r" % e

In [21]:
def gnd_auswertung(matches):
    if type(matches) == list:
        return [match['idn'] for match in matches if match['datum_match'] == True]

In [22]:
# idn abfrage starten und ergebnisse in neue spalte matches schreiben
df['matches'] = df.apply(gnd_abfrage, axis=1)

KeyboardInterrupt: 

In [24]:
# auswertung starten und matchende idns in neue spalte idns schreiben
df['idns'] = df.matches.apply(gnd_auswertung)

In [61]:
def pica_schreiben(row):
    pica = f"""0500 Qd
0501 Text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
1100 {row.tod[-4:]}$n[nicht nach {row.tod[-4:]}]
1130 TB-papier
1130 !040445224!
1131 !041466098!
1132 a1-analog;f1-text;f2-blatt;v-cont
1500 /1ger
1700 /1XA-DE
3100 !000073369!$BZusammenstellender$4com
4000 Ehrenamtsakte {row.vorname} {row.nachname}
4019 Ehrenamtsakte$Bobja
4060 {row.umfang}
4105 !1061153886!
4700 |BSM|
5100 !000073369!
"""
    if len(row.idns) > 2:
        pica += f"5101 !{row.idns[2:-2]}!\n"
    
    pica += f"""5590 [Objektgattung]
5590 !040009254!
7001 {now.strftime('%d-%m-%y')} : x
4800 !009033645!
6710 !1265505004!$l{row.signatur.split(",")[1]}
7100 HA/BV 93,{row.signatur.split(",")[1]} @ m
7109 !!DBSM/S!! ; HA/BV 93,{row.signatur.split(",")[1]}
\t\n"""
    return pica


In [55]:
now = datetime.now()
for index, row in df[:5].iterrows():
    print(pica_schreiben(row))

AttributeError: 'str' object has no attribute 'str'

In [62]:
now = datetime.now()
with open(f"../dat/ehrenamstkartei-{now.strftime('%y-%m-%d-%H-%M-%S')}.dat","w") as f:
    with open(f"../dat/ehrenamstkartei-recent.dat", 'w') as f2:
        for index, row in df[:100].iterrows():
            f.write(pica_schreiben(row))
            f2.write(pica_schreiben(row))

with open(f"../dat/ehrenamstkartei-recent-sample.dat", 'w') as f:
    for index, row in df[:100].sample(5).iterrows():
        f.write(pica_schreiben(row))

TypeError: 'NAType' object is not subscriptable

In [52]:
df

,signatur,nachname,vorname,geburt,tod,umfang,matches,idns
0,"HA/BV 93,1",Adrianwild,Werner,21.04.1915,18.08.2001,17 Seiten,[],[]
1,"HA/BV 93,2",Ammann,Egon,09.10.1941,09.08.2017,4 Seiten,"[{'idn': '10906769X', 'datum_match': True}]",['10906769X']
2,"HA/BV 93,3",Anrich,Gerold,24.11.1942,04.10.2013,21 Seiten,"[{'idn': '120243695', 'datum_match': False}]",[]
3,"HA/BV 93,4",Appel,Rolf,16.06.1920,03.04.2019,16 Seiten,"[{'idn': '104609664', 'datum_match': False}, {'idn': '116316357', 'datum_match': False}, {'idn': '173874452', 'datum_match': False}]",[]
4,"HA/BV 93,5",Arndt,Martin,02.10.1917,02.03.2005,12 Seiten,"[{'idn': '1033079022', 'datum_match': False}, {'idn': '1258417111', 'datum_match': False}, {'idn': '1017999198', 'datum_match': False}, {'idn': '13286410X', 'datum_match': False}, {'idn': '124861393', 'datum_match': False}, {'idn': '115655514', 'datum_match': False}, {'idn': '115563040', 'datum_match': False}, {'idn': '12019225X', 'datum_match': False}, {'idn': '130040207', 'datum_match': False}, {'idn': '121999605X', 'datum_match': False}, {'idn': '103140452X', 'datum_match': False}, {'idn': '1046090283', 'datum_match': False}, {'idn': '1235620085', 'datum_match': False}]",[]
...,...,...,...,...,...,...,...,...
567,<NA>,Zeller,Bernhard,19.09.1919,07.09.2008,<NA>,"[{'idn': '1123217912', 'datum_match': False}, {'idn': '143394525', 'datum_match': False}, {'idn': '173825303', 'datum_match': False}, {'idn': '1243886684', 'datum_match': False}, {'idn': '118772465', 'datum_match': True}, {'idn': '173693156', 'datum_match': False}]",['118772465']
568,<NA>,Zill,Wolfgang,10.12.1928,16.06.1986,<NA>,"[{'idn': '1062130359', 'datum_match': False}]",[]
569,<NA>,Zipp,Marianne,14.03.1920,01.10.1983,<NA>,[],[]
570,<NA>,Zirk,Heinz,21.01.1947,09.08.2012,<NA>,[],[]


In [53]:
"['10906769X']"[2:-2]

'10906769X'